In [ ]:
%conf 
numRows=5
streaming=false

# 1. Extract static data

In [ ]:
{
  "type": "DelimitedExtract",
  "name": "extract initial table",
  "environments": ["dev", "test"],
  "inputURI": "s3a://"${ETL_CONF_DATALAKE_LOC}"/app_code/data/initial_contacts.csv",
  "schemaURI":"s3a://"${ETL_CONF_DATALAKE_LOC}"/app_code/meta/contact_meta_0.json",
  "outputView": "initial_raw",            
  "delimiter": "Comma",
  "header": false,
  "quote": "None"
}

In [ ]:
{
  "type": "TypingTransform",
  "name": "apply table schema 0",
  "environments": ["dev", "test"],
  "schemaURI": "s3a://"${ETL_CONF_DATALAKE_LOC}"/app_code/meta/contact_meta_0.json",
  "inputView": "initial_raw",            
  "outputView": "initial_typed",
  "numPartitions": 1
  "persist":true
}

In [ ]:
{
  "type": "DelimitedExtract",
  "name": "extract updated data",
  "environments": ["dev", "test"],
  "inputURI": "s3a://"${ETL_CONF_DATALAKE_LOC}"/app_code/data/update_contacts.csv",
  "schemaURI":"s3a://"${ETL_CONF_DATALAKE_LOC}"/app_code/meta/wcontact_meta_0.json",
  "outputView": "delta_raw",            
  "delimiter": "Comma",
  "header": false,
  "quote": "None"
}

In [ ]:
{
  "type": "TypingTransform",
  "name": "apply table schema 0",
  "environments": ["dev", "test"],
  "schemaURI": "s3a://"${ETL_CONF_DATALAKE_LOC}"/app_code/meta/contact_meta_0.json",
  "inputView": "delta_raw",            
  "outputView": "delta_typed",
  "numPartitions": 1
  "persist":true
}

# 2. Turn on Spark Streaming

In [ ]:
%conf 
streaming=true
streamingDuration=30

# 2.1 Convert static data to stream
- Initial stream = Initial dataset
- Delta stream = Incremental dataset

In [ ]:
{
  "type": "RateExtract",
  "name": "create a streaming source",
  "environments": [
    "production",
    "test"
  ],
  "outputView": "initial_stream",
  "numPartitions": 1,
  "rowsPerSecond": 5
}

In [ ]:
%sql outputView="stream_a" name="simulate a stream" sqlParams=input_table=initial_typed,stream_table=initial_stream numPartitions=1

SELECT *
FROM ${stream_table} 
INNER JOIN ${input_table}
ON ${input_table}._index = ${stream_table}.value

In [ ]:
{
  "type": "RateExtract",
  "name": "create a streaming source",
  "environments": [
    "production",
    "test"
  ],
  "outputView": "delta_stream",
  "numPartitions": 1,
  "rowsPerSecond": 5
}

In [ ]:
%sql outputView="stream_b" name="simulate b stream" sqlParams=input_table=delta_typed,stream_table=delta_stream numPartitions=1

SELECT *
FROM ${stream_table} 
INNER JOIN ${input_table}
ON ${input_table}._index = ${stream_table}.value

In [ ]:
%sql outputView="join_streams" name="join two streams"

SELECT
    initial.id as initial_id,
    initial.name as initial_name,
    initial.email as initial_email,
    initial.state as initial_state,
    delta.email as delta_email,
    delta.state as delta_state
FROM stream_a initial
INNER JOIN stream_b delta
ON initial.id = delta.id
where initial.email<>delta.email or initial.state<>delta.state
